# 네이버금융에서 주식데이터 불러오기

## 1. 네이버 금융에서 주식 데이터 불러오기

출처 - https://excelsior-cjh.tistory.com/109


In [1]:
import pandas as pd
df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]

#종목코드는 6자리이므로 6자리로 맞춰준다.
df['종목코드'] = df['종목코드'].map('{:06d}'.format)

# 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다. 
df = df[['회사명', '종목코드']]

#한글로된 컬럼명을 영어로 바꿔준다.
df = df.rename(columns={'회사명':'name', '종목코드':'code'})
df.sort_values('name').head()

# 원하는 종목의 코드를 가져오는 함수
def get_url(item_name, data):
    code = data[data['name'] == item_name]['code']
    url = ('http://finance.naver.com/item/sise_day.nhn?code=' + code.iloc[0])
    return url

url = get_url('카카오', df)

#일자 데이터를 담을 df라는 DataFrame 정의
stock = pd.DataFrame()

# 1페이지에서 476페이지까지 가져오기
import timeit
start = timeit.default_timer()

for page in range(1, 476):
    pg_url = '{url}&page={page}'.format(url=url, page=page)
    stock = stock.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)
    
stock = stock.dropna()

stop = timeit.default_timer()
print(stop - start)

stock = stock.sort_values(by=['날짜'])

82.48588846333382


In [2]:
stock.head()

,날짜,종가,전일비,시가,고가,저가,거래량
7123,1999.12.06,75600.0,8100.0,75600.0,75600.0,75600.0,2279.0
7122,1999.12.07,84600.0,9000.0,84600.0,84600.0,84600.0,1858.0
7121,1999.12.08,94700.0,10100.0,94700.0,94700.0,94700.0,530.0
7120,1999.12.09,106000.0,11300.0,106000.0,106000.0,106000.0,303.0
7119,1999.12.10,118500.0,12500.0,118500.0,118500.0,118500.0,919.0
